# Session 03: Intro to Bokeh Plotting on the Science Platform

<br>Owner(s): **Keith Bechtol** ([@bechtol](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@bechtol))
<br>Last Verified to Run: **2020-05-19**
<br>Verified Stack Release: **w_2020_19**

This notebook introduces the concept of linked visualizations. We begin with a few simple examples using abstract data, largely based on the Bokeh documentation:
* https://docs.bokeh.org/en/latest/docs/user_guide/interaction/linking.html
* https://docs.bokeh.org/en/latest/docs/gallery.html#notebook-examples
* https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html

In the second part, we apply the linked brushing technique to a coadd catalog from HSC.

In [ ]:
## Abstract Examples of Linking

In [1]:
from bokeh.io import output_notebook, show 
from bokeh.layouts import gridplot
from bokeh.plotting import figure

# Need this line to display bokeh plots inline in the notebook
output_notebook()

Loading BokehJS ...

In [2]:
import numpy as np

x = np.arange(11)
y0 = x
y1 = 10 - x
y2 = np.fabs(x - 5)

# create a new plot
s1 = figure(plot_width=250, plot_height=250, title=None)
s1.circle(x, y0, size=10, color="navy", alpha=0.5)

# create a new plot and share both ranges
s2 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, y_range=s1.y_range, title=None)
s2.triangle(x, y1, size=10, color="firebrick", alpha=0.5)

# create a new plot and share only one range
s3 = figure(plot_width=250, plot_height=250, x_range=s1.x_range, title=None)
s3.square(x, y2, size=10, color="olive", alpha=0.5)

p = gridplot([[s1, s2, s3]], toolbar_location='left')

# show the results
show(p)

In [12]:
from bokeh.models import ColumnDataSource

x = np.arange(-20, 21)
y0 = np.fabs(x)
y1 = x**2

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_zoom,box_select,lasso_select,reset"

# create a new plot and add a renderer
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

In [13]:
from bokeh.layouts import column
from bokeh.models import Slider
from bokeh.plotting import figure, show

plot = figure(plot_width=400, plot_height=400)
r = plot.circle([1,2,3,4,5,], [3,2,5,6,4], radius=0.2, alpha=0.5)

slider = Slider(start=0.05, end=1, step=0.01, value=0.2)
slider.js_link('value', r.glyph, 'radius')

show(column(plot, slider))

## Brushing and Linking Example with HSC Data

The cell below takes about a minute to run.

Let's extend the ideas above with an application to astronomical data. We'll use the same HSC coadd catalog from the intro to matplotlib notebook.

In [15]:
%%time

import utils
data = utils.getData()

HSC-G 9615 0,0
3986
HSC-G 9615 0,1
7888
HSC-G 9615 0,2
8180
HSC-G 9615 1,0
8942
HSC-G 9615 1,1
17085
HSC-G 9615 1,2
20062
HSC-G 9615 2,0
9074
HSC-G 9615 2,1
22275
HSC-G 9615 2,2
18565
HSC-R 9615 0,0
3986
HSC-R 9615 0,1
7888
HSC-R 9615 0,2
8180
HSC-R 9615 1,0
8942
HSC-R 9615 1,1
17085
HSC-R 9615 1,2
20062
HSC-R 9615 2,0
9074
HSC-R 9615 2,1
22275
HSC-R 9615 2,2
18565
HSC-I 9615 0,0
3986
HSC-I 9615 0,1
7888
HSC-I 9615 0,2
8180
HSC-I 9615 1,0
8942
HSC-I 9615 1,1
17085
HSC-I 9615 1,2
20062
HSC-I 9615 2,0
9074
HSC-I 9615 2,1
22275
HSC-I 9615 2,2
18565
CPU times: user 42.9 s, sys: 4.23 s, total: 47.1 s
Wall time: 47.1 s


Define a few variables for plotting.

In [16]:
# Variables to plot
concentration = data['HSC-I']['psf_mag'] - data['HSC-I']['cm_mag']
gr = data['HSC-G']['cm_mag'] - data['HSC-R']['cm_mag']
ri = data['HSC-R']['cm_mag'] - data['HSC-I']['cm_mag']
i = data['HSC-I']['cm_mag']

Now create the two-panel figure with brushing and linking. We pass our variables into the `ColumnDataSource` object, and link the two panels by pointing both to same source.

**Exercise:** Use the selection tools to investigate different source populations according to their morphology and colors.

In [17]:
source = ColumnDataSource(data=dict(c=concentration, gr=gr, ri=ri, i=i))

TOOLS = "box_zoom,box_select,lasso_select,reset"

# More attractive colors
kwargs = {'selection_fill_color':'steelblue', 
          'selection_line_color': 'steelblue',
          'nonselection_fill_color': 'silver', 
          'nonselection_line_color': 'silver'}

# Flux-Concentration
left = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
left.circle('i', 'c', source=source, **kwargs)
left.xaxis.axis_label = 'i (mag)'
left.yaxis.axis_label = 'concentration'

# Color-color
right = figure(tools=TOOLS, plot_width=300, plot_height=300, title=None)
right.circle('gr', 'ri', source=source, **kwargs)
right.xaxis.axis_label = 'r - i'
right.yaxis.axis_label = 'g - r'

p = gridplot([[left, right]])

show(p)

To see more examples of interactive plotting with Bokeh, HoloViews, and Datashader see the following Stack Club [notebook](https://github.com/LSSTScienceCollaborations/StackClub/blob/master/Visualization/bokeh_holoviews_datashader.ipynb). 